## Iowa liquor data preparation

This notebook simplifies the Iowa Liquor Sales dataset to make it easier to demo.

In [1]:
from google.cloud import bigquery

client = bigquery.Client()

### Retrieve dataset as dataframe

In [2]:
dataset_uri = "bigquery-public-data.iowa_liquor_sales_forecasting.2020_sales_train"

In [3]:
sql = f"""
    SELECT *
    FROM `{dataset_uri}`
"""

df = client.query(sql).to_dataframe()

### Clean data

In [4]:
df = df.sort_values("date")
df = df.dropna()

In [5]:
df.columns

Index(['date', 'store_name', 'city', 'zip_code', 'county', 'sale_dollars'], dtype='object')

In [6]:
df.head()

,date,store_name,city,zip_code,county,sale_dollars
31206,2020-01-02,Best Food Mart 3 LLC,Des Moines,50311.0,POLK,317.04
33897,2020-01-02,Hy-Vee #3 / BDI / Des Moines,Des Moines,50320.0,POLK,65864.59
23410,2020-01-02,KUM & GO #292 / Ankeny,Ankeny,50021.0,POLK,3428.22
45898,2020-01-02,Tobacco Shop / Arnolds Park,Arnolds Park,51331.0,DICKINSON,5006.09
24009,2020-01-02,Hy-Vee Gas #1 / Ankeny,Ankeny,50023.0,Polk,330.74


In [7]:
df["county_and_city"] = df.county + " - " + df.city

### Only keep rows from counties and cities with the most rows

This is to reduce the dataset for demo purposes

In [8]:
group = "county_and_city"

# Get zip_code_store_name with most rows
group_n_largest = df.groupby(group).sale_dollars.count().nlargest(12)

group_n_largest

county_and_city
POLK - Des Moines              4623
LINN - Cedar Rapids            3002
SCOTT - Davenport              2371
BLACK HAWK - Waterloo          1838
POLK - West Des Moines         1599
POTTAWATTA - Council Bluffs    1525
WOODBURY - Sioux City          1462
DUBUQUE - Dubuque              1369
JOHNSON - Iowa City            1308
STORY - Ames                   1286
POLK - Ankeny                  1183
POLK - Urbandale                953
Name: sale_dollars, dtype: int64

In [9]:
group_n_largest_names = group_n_largest.reset_index()[group].tolist()

group_n_largest_names

['POLK - Des Moines',
 'LINN - Cedar Rapids',
 'SCOTT - Davenport',
 'BLACK HAWK - Waterloo',
 'POLK - West Des Moines',
 'POTTAWATTA - Council Bluffs',
 'WOODBURY - Sioux City',
 'DUBUQUE - Dubuque',
 'JOHNSON - Iowa City',
 'STORY - Ames',
 'POLK - Ankeny',
 'POLK - Urbandale']

In [10]:
df_filtered = df[df[group].isin(group_n_largest_names)]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plot sales
fig = plt.gcf()
fig.set_size_inches(24, 12)

In [ ]:
sns.relplot(
    data=df_filtered,
    x="date",
    y="sale_dollars",
    # hue="project_name_and_service",
    row=group,
    height=5,
    aspect=4,
    kind="line",
    # facet_kws={'sharey': False, 'sharex': True},
    errorbar=None,
)

In [ ]:
df_filtered.to_csv("iowa_liquor_sales.csv", index=False)